In [1]:
import os
os.chdir('..')

In [2]:
import pandas as pd
import numpy as np

import scipy.optimize as sco

import warnings
warnings.filterwarnings('ignore')

import sklearn.cluster as cl
from sklearn import mixture

data_folder = '../data/'
path = data_folder + 'GEV_SM/swissmetro.dat'

from classes.MNLogit import *
from helpers.data import *
from helpers.algos import *
from helpers.models import *
import copy

import matplotlib.pyplot as plt

import time
import pickle

import hyperopt
from hyperopt.pyll.base import scope
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

import seaborn as sns

import numdifftools as nd

sns.set(font_scale=1.5)

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

seed = 1234

In [3]:
model = load_model(path, 'norm')
x = np.zeros(len(model.params))

In [4]:
%%time
resbfgs = sco.minimize(model.negloglikelihood, x, method='BFGS', tol=1e-8, jac=model.neg_grad)

CPU times: user 5.37 s, sys: 64 ms, total: 5.43 s
Wall time: 10.6 s


In [5]:
resbfgs.x

array([ 0.786179  ,  0.98264685, -0.65596648, -1.04933718, -0.68768718,
       -1.05748236, -0.8000895 , -1.44306552, -1.45576372, -1.79689044])

In [6]:
resbfgs.fun

0.7908058095403135

# SBFGS

In [7]:
nbr = 20
draws = 10

res = {}

In [8]:
dct = {}  

start = time.time()
epochs, xs, lls  = bfgs(model, x, nbr, 'eye', False)
stop = time.time()

dct['epochs'] = epochs
dct['lls'] = lls
dct['times'] = stop-start

res['BFGS-eye'] = dct

Start BFGS for 20 epochs (20 iterations)


In [9]:
dct = {}  

start = time.time()
epochs, xs, lls  = bfgs(model, x, nbr, 'hessian', False)
stop = time.time()

dct['epochs'] = epochs
dct['lls'] = lls
dct['times'] = stop-start

res['BFGS-hess'] = dct

Start BFGS for 20 epochs (20 iterations)


In [10]:
dct = {}

lls = []
times = []
for d in range(draws):
    
    start = time.time()
    ep, x_val, ll = res_bfgs(model, x, nbr, 100)
    stop = time.time()
    
    times.append(stop-start)
    lls.append(ll)
    
dct['epochs'] = np.array(ep)
dct['lls'] = np.array(lls)
dct['times'] = np.array(times)

res['RES_BFGS-100'] = dct    

Start Regularized stochastic BFGS for 20 epochs (1808 iterations)
Start Regularized stochastic BFGS for 20 epochs (1808 iterations)
Start Regularized stochastic BFGS for 20 epochs (1808 iterations)
Start Regularized stochastic BFGS for 20 epochs (1808 iterations)
Start Regularized stochastic BFGS for 20 epochs (1808 iterations)
Start Regularized stochastic BFGS for 20 epochs (1808 iterations)
Start Regularized stochastic BFGS for 20 epochs (1808 iterations)
Start Regularized stochastic BFGS for 20 epochs (1808 iterations)
Start Regularized stochastic BFGS for 20 epochs (1808 iterations)
Start Regularized stochastic BFGS for 20 epochs (1808 iterations)


In [11]:
dct = {}

lls = []
times = []
for d in range(draws):
    
    start = time.time()
    ep, x_val, ll = res_bfgs(model, x, nbr, 1000)
    stop = time.time()
    
    times.append(stop-start)
    lls.append(ll)
    
dct['epochs'] = np.array(ep)
dct['lls'] = np.array(lls)
dct['times'] = np.array(times)

res['RES_BFGS-1000'] = dct    

Start Regularized stochastic BFGS for 20 epochs (181 iterations)
Start Regularized stochastic BFGS for 20 epochs (181 iterations)
Start Regularized stochastic BFGS for 20 epochs (181 iterations)
Start Regularized stochastic BFGS for 20 epochs (181 iterations)
Start Regularized stochastic BFGS for 20 epochs (181 iterations)
Start Regularized stochastic BFGS for 20 epochs (181 iterations)
Start Regularized stochastic BFGS for 20 epochs (181 iterations)
Start Regularized stochastic BFGS for 20 epochs (181 iterations)
Start Regularized stochastic BFGS for 20 epochs (181 iterations)
Start Regularized stochastic BFGS for 20 epochs (181 iterations)


In [12]:
with open('../data/SBFGS2.p', 'wb') as outfile:
    pickle.dump(res, outfile)

In [16]:
with open('../data/SBFGS2.p', 'rb') as infile:
    res = pickle.load(infile)

colors = {'RES_BFGS-100': (232/255,164/255,29/255), 
          'RES_BFGS-1000': (0/255,152/255,205/255)}

labels = {
    'BFGS-eye': 'BFGS (Identity start)',
    'BFGS-hess': 'BFGS (Hessian start)',
    'RES_BFGS-100': 'RES-BFGS (batch size: 100)',
    'RES_BFGS-1000': 'RES-BFGS (batch size: 1000)'
}

plt.figure(figsize=(5,3), frameon=False)

sns.set_context("paper")
sns.set(font_scale = 1.3)
sns.set_style("white", {
    "font.family": "sans-serif",
    "font.serif": ['Helvetica'],
    "font.scale": 2
})
sns.set_style("ticks", {"xtick.major.size": 4,
                        "ytick.major.size": 4})

ax = plt.subplot(111)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

ax.plot([res['BFGS-eye']['epochs'][0], res['BFGS-eye']['epochs'][-1]], 
        [-resbfgs.fun, -resbfgs.fun], 'r--', 
        label='Optimal log-likelihood')

ax.plot(res['BFGS-eye']['epochs'], res['BFGS-eye']['lls'], 'k', label=labels['BFGS-eye'])
ax.plot(res['BFGS-hess']['epochs'], res['BFGS-hess']['lls'], '--k', label=labels['BFGS-hess'])


for key in ['RES_BFGS-100', 'RES_BFGS-1000']:
    
    epochs = res[key]['epochs']
    
    plus = []
    minus = []
    avg = []
    
    vals = res[key]['lls']
    
    for i in range(vals.shape[1]):
        avg.append(np.mean(vals[:,i]))
        minus.append(np.percentile(vals[:,i], 5))
        plus.append(np.percentile(vals[:,i], 95))
    
    ax.plot(epochs, avg, linestyle='-', color=colors[key] , label=labels[key])
    ax.fill_between(epochs, plus, minus, color=colors[key] , alpha=0.5)

plt.xlabel('Epoch')
plt.ylabel('Normalized log-likelihood ($\\bar{\\mathcal{L}}$)')
leg = ax.legend(frameon=True)
max_ep = 20

ax.set_xlim([-0.05, max_ep])

plt.savefig('../figures/SBFGS{}.pdf'.format(max_ep), bbox_inches='tight')

In [14]:
for algo in res.keys():
    
    avg = {}

    lls = res[algo]['lls']
    
    try:
        tmp = lls[:,-1]

        avg = np.mean(tmp)
    except:
        avg = lls[-1]
        
    print("  LL for {}: {:.6f}".format(algo, avg))

  LL for BFGS-eye: -0.963458
  LL for BFGS-hess: -0.806245
  LL for RES_BFGS-100: -0.796492
  LL for RES_BFGS-1000: -0.837937
